In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Python Version**

In [2]:
!python --version

Python 3.10.12


# **GPU Checking**

In [3]:
!nvidia-smi

Sat Feb 17 14:17:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    print(f"GPU: {device}")
else:
    print("No GPU available.")

GPU: Tesla T4


# **Importing Dependancies**

In [5]:
!pip install ultralytics==8.0.28

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5432.0/8062.4 GB disk)


In [6]:
from ultralytics import YOLO
from IPython.display import display, Image

In [7]:
import torch
print(torch.version.cuda)
torch.cuda.is_available()

11.8


True

# **Train**

In [ ]:
!yolo task=segment mode=train model=/kaggle/input/segment-model/yolov8s-seg.pt data=/kaggle/input/fydp-yolov8-yaml/data.yaml  epochs=5  save=True  imgsz=640 batch=16

Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=/kaggle/input/segment-model/yolov8s-seg.pt, data=/kaggle/input/fydp-yolov8-yaml/data.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=F

In [ ]:
!yolo  task=segment  mode=val  model=/kaggle/working/runs/segment/train/weights/best.pt   data=/kaggle/input/fydp-yolov8-yaml/data.yaml  conf=0.5  source="/kaggle/input/fydp-yolov8/FYDP_Yolov8/test/images"

# **Train Result/Epoch**

In [ ]:
from tabulate import tabulate
import csv

csv_file_path = '/kaggle/working/runs/segment/train/results.csv'

with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    data = list(csv_reader)

print(tabulate(data, headers=header, tablefmt='grid'))

# **Result Graph**

In [ ]:
from IPython.display import display, Image

Image(filename="/kaggle/working/runs/segment/train/results.png", width=600)

# **Confusion Matrics**

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/confusion_matrix.png", width=600)

# **F1 Score Curve**

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/BoxF1_curve.png", width=600)

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/MaskF1_curve.png", width=600)

# **Precision Curve**

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/MaskP_curve.png", width=600)

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/BoxP_curve.png", width=600)

# **Recall Curve**

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/BoxR_curve.png", width=600)

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/MaskR_curve.png", width=600)

# **Precision-Recall Curve**

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/BoxPR_curve.png", width=600)

In [ ]:
Image(filename="/kaggle/working/runs/segment/train/MaskPR_curve.png", width=600)

# **Test**

In [ ]:
!yolo task=segment mode=predict save_txt=True model=/kaggle/working/runs/segment/train/weights/best.pt conf=0.25 source="/kaggle/input/fydp-yolov8/FYDP_Yolov8/test/images"

# **Making Zip & Download Prediction**

In [ ]:
import shutil
import zipfile

source_folder = '/kaggle/working/runs'
zip_filename = '/kaggle/working/runs'
shutil.make_archive(zip_filename, 'zip', source_folder)

In [ ]:
image = cv2.imread('/kaggle/working/runs/segment/predict/5e15e99f-258a-4958-a55e-8879d15d5d72_jpg.rf.2c26700c3d41d638b2fb6e6e65008a51.jpg')

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('off')  
plt.show()

# **Working Directory Cleaner(If needed)**

In [ ]:
#!rm -rf /kaggle/working/*